# Test Stochastic Program implementation

In [1]:
import os
os.chdir('..') # move to the root directory (from dev)

In [2]:
import numpy as np
from utils import build_schema
from linmodel import LinProgModel
from citylearn.citylearn import CityLearnEnv

In [3]:
# set up parameters
dataset_dir = os.path.join('data','processed') # dataset directory

base_params = {
    'data_dir_path': os.path.join('data','processed'),
    'weather_data_path': 'weather.csv',
    'carbon_intensity_data_path': 'carbon_intensity.csv',
    'pricing_data_path': 'pricing.csv',
    'schema_name': 'schema_build_test',
    'battery_efficiencies': [0.95,0.95,0.95],
    'battery_energy_capacities': None,
    'battery_power_capacities': None,
    'pv_power_capacities': None
}

In [4]:
# set up LP costs
cost_dict = {
    'carbon': 5e-1,
    'battery': 1000, #1e3,
    'solar': 1000, #2e3,
    'grid_capacity': 50e-2*365/0.95,
    'grid_excess': 100e-2*365/0.95,
    'opex_factor': 20,
    'battery_power_ratio': 0.4
}

In [5]:
# set up load profiles
building_year_sets = [
    [(40,2015),(4,2012)],
    [(40,2016),(4,2013)],
    [(40,2014),(4,2015)],
]

In [6]:
# set up stochastic program
envs = []

for m, lys in enumerate(building_year_sets):
    # build schema
    params = base_params.copy()
    params['building_names'] = [f'TB{i}' for i in range(len(lys))]
    params['load_data_paths'] = [f'ly_{b}-{y}.csv' for b,y in lys]
    params['schema_name'] = f'SP_test_schema_{m}'
    schema_path = build_schema(**params)

    # Initialise CityLearn environment object.
    envs.append(CityLearnEnv(schema=schema_path))

    if m == 0: # initialise lp object
        lp = LinProgModel(env=envs[m])
    else:
        lp.add_env(env=envs[m])

In [7]:
lp.set_time_data_from_envs(t_start=3360,tau=24*7*4)
lp.generate_LP(cost_dict,design=True)
lp.set_LP_parameters()

In [8]:
lp_results = lp.solve_LP(verbose=True,ignore_dpp=True)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) May 02 03:40:38 PM: Your problem has 14117 variables, 27 constraints, and 12102 parameters.
(CVXPY) May 02 03:40:38 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 02 03:40:38 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 02 03:40:38 PM: Compiling problem (target solver=SCIPY).
(CVXPY) May 02 03:40:38 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIPY
(CVXPY) May 02 03:40:38 PM: Applying reduction Dcp2Cone
(CVXPY) May 02 03:40:38 PM: Applying reduction CvxAttr2Const

/Users/mal84/opt/miniconda3/envs/BD-VOI/lib/python3.8/site-packages/cvxpy/reductions/solvers/conic_solvers/scipy_conif.py:99: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (StrictVersion(scipy.__version__) < StrictVersion('1.6.1')):


Presolve : Reductions: rows 28221(-10094); columns 14117(-6); elements 76566(-10112)
INFO   : Solving the presolved LP
INFO   : Scaling: Improvement factor is 64.49 >= 1 so scale LP
INFO   : Basis condition estimate of           1 is within the tolerance of 1e+14
INFO   : Using dual simplex solver - serial
       Iteration        Objective     Infeasibilities num(sum)
DuPh2          0     0.0000000000e+00 Pr: 8064(522415); Du: 0(2.92889e-09)
DuPh2        455     3.8083408419e+03 Pr: 8020(487704); Du: 0(2.70424e-09)
DuPh2        923     1.0185122299e+04 Pr: 7874(502225); Du: 0(1.37704e-08)
DuPh2       1297     4.7138394678e+05 Pr: 7933(528617); Du: 0(9.52684e-09)
DuPh2       1677     6.7775220752e+05 Pr: 7661(462880); Du: 0(9.44757e-09)
DuPh2       2089     6.9486452333e+05 Pr: 7642(453558); Du: 0(4.0979e-08)
DuPh2       2496     7.1445994053e+05 Pr: 7599(434608); Du: 0(9.2445e-09)
DuPh2       2932     7.3527472245e+05 Pr: 7569(413286); Du: 0(9.13917e-09)
DuPh2       3363     7.56726060

In [9]:
print(lp_results['objective'])
print(lp_results['objective_contrs'])
print(lp_results['battery_capacities'])
print(lp_results['solar_capacities'])
print(lp_results['grid_capacity'])

1597232.5294267158
[332724.6975529466, 98288.6860993209, 55811.13146790348, 705205.4350518563, 31717.002736893097, 373485.5765177954]
[[-0.        ]
 [31.71700274]]
[[149.68145955]
 [223.80411697]]
183.54662008199


In [10]:
for m in range(len(envs)):
    print(np.max(lp_results['e_grids'][m]))

205.33589743589746
183.54662008199006
183.54662008199
